In [3]:
import pandas as pd
import duckdb

df = duckdb.sql("SELECT * FROM read_parquet('C:\\Users\\nikhi\\Documents\\Projects\\e2e-market-attribution-and-budget-optimization\\data\\bronze\\raw_impressions.parquet')").fetchdf()
df.head()

,impression_id,timestamp,uid,campaign,conversion,conversion_timestamp,conversion_id,attribution,click,click_pos,...,time_since_last_click,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9
0,1,0,20073966,22589171,0,-1,-1,0,0,-1,...,-1,5824233,9312274,3490278,29196072,11409686,1973606,25162884,29196072,29196072
1,2,2,24607497,884761,0,-1,-1,0,0,-1,...,423858,30763035,9312274,14584482,29196072,11409686,1973606,22644417,9312274,21091111
2,3,2,28474333,18975823,0,-1,-1,0,0,-1,...,8879,138937,9312274,10769841,29196072,5824237,138937,1795451,29196072,15351056
3,4,3,7306395,29427842,1,1449193,3063962,0,1,0,...,-1,28928366,26597095,12435261,23549932,5824237,1973606,9180723,29841067,29196072
4,5,3,25357769,13365547,0,-1,-1,0,0,-1,...,-1,138937,26597094,31616034,29196072,11409684,26597096,4480345,29196072,29196072


# 1. Campaign Performance

## 1.1 Which campaigns have the highest conversion rates?

## 1.2 What's the cost distribution across campaigns?
- Identify high-spend vs. low-spend campaign
- Check for outliers (campaigns with abnormally high CPM)

## 1.3 Time-based patterns:
- Do conversions spike on weekends?
- Which hour of the day has best conversion rate?
- Is there a day-of-week effect?

# 2. User Behavior

## 2.1 What's the distribution of user journey lengths?
- What % of users convert on first impression vs. multi-touch?
- What's the median journey length for converters vs. non-converters?

## 2.2 Time-to-conversion analysis:
- Among converters, what's the median time from first impression to conversion?
- Are there patterns (e.g., most convert within 3 days)?

## 2.3 Multi-touch patterns:
- What % of conversions happen after seeing 1, 2, 3, 4+ campaigns?
- Do users who see diverse campaigns convert more?

# 3. Economic Analysis

## 3.1 Understanding the `cpo` Metric (Critical Analysis)

**Key insight:** The `cpo` column is **only meaningful when attribution=1** (actual cost-per-order). For other rows, it is a placeholder/predicted value (bidding signal). Silver layer splits this into `cost_per_order_actual` and `cost_per_order_predicted`.

In [ ]:
from pathlib import Path

# Silver path (run ETL pipeline first)
silver_path = Path("data/silver/clean_impressions.parquet")
if not silver_path.exists():
    silver_path = Path("../data/silver/clean_impressions.parquet")
silver_path_str = str(silver_path.resolve()).replace("\\", "/")

# Corrected analysis: cpo is only actual cost when attribution=1
conn = duckdb.connect()
cpo_analysis = conn.execute(f"""
    SELECT 
        CASE 
            WHEN attribution THEN 'Attributed (Actual CPO)'
            WHEN conversion THEN 'Converted but Not Attributed'
            ELSE 'No Conversion'
        END as row_type,
        COUNT(*) as row_count,
        AVG(cost_per_order_actual) as avg_actual_cpo,
        AVG(cost_per_order_predicted) as avg_predicted_cpo,
        MIN(cost_per_order_predicted) as min_cpo,
        MAX(cost_per_order_predicted) as max_cpo,
        STDDEV(cost_per_order_predicted) as stddev_cpo
    FROM read_parquet('{silver_path_str}')
    GROUP BY row_type
    ORDER BY 
        CASE 
            WHEN row_type = 'Attributed (Actual CPO)' THEN 1
            WHEN row_type = 'Converted but Not Attributed' THEN 2
            ELSE 3
        END
""").fetchdf()

print("\n=== CPO Column Analysis by Row Type ===")
print(cpo_analysis)

# For attributed conversions, cost_per_order_actual represents ACTUAL cost-per-order
# For other rows, cost_per_order_predicted is a predicted/placeholder value
attributed_rows = cpo_analysis[cpo_analysis["row_type"] == "Attributed (Actual CPO)"]
if len(attributed_rows) > 0:
    print(f"\nAttributed (Actual CPO) avg: {attributed_rows['avg_actual_cpo'].iloc[0]:.4f}")

# Calculate actual CPA from cost column
actual_cpa = conn.execute(f"""
    SELECT 
        SUM(cost) / NULLIF(SUM(CAST(attribution AS INTEGER)), 0) as actual_cpa_from_cost,
        AVG(CASE WHEN attribution THEN cost_per_order_actual END) as avg_cpo_for_attributed
    FROM read_parquet('{silver_path_str}')
""").fetchdf()

print("\n=== Actual CPA vs CPO Comparison ===")
print(f"Actual CPA (sum of costs / attributed conversions): ${actual_cpa['actual_cpa_from_cost'].iloc[0]:.4f}")
print(f"Average CPO value (for attributed rows): ${actual_cpa['avg_cpo_for_attributed'].iloc[0]:.4f}")
print("\n(These should be similar but not identical: cpo is per-conversion, CPA is aggregate)")

## 3.1 What's the overall ROAS (Revenue On Ad Spend)?

## 3.2 Which campaigns are profitable?
- Calculate campaign-level ROAS (you'll use attribution models later, but start with last-touch assumption)

## 3.3 Cost efficiency:
- What's the Cost Per Click (CPC)?
- What's the Cost Per Acquisition (CPA)?
- Are there diminishing returns (does doubling spend on a campaign double conversions)?

# 4. Data Quality Insights

## 4.1 Conversion without click anomaly:
- How many conversions happened without a click? (Should investigate if > 5%)

## 4.2 User-level data sufficiency:
- Do we have enough multi-touch users to build attribution models?
- What % of users have only 1 touchpoint? (These won't help MTA)

## 4.3 Temporal coverage:
- How many days does the dataset span?
- Are there gaps in data? (Missing days could bias analysis)